In [127]:
import numpy as np
import yaml
import matplotlib.pyplot as plt
import csv

In [128]:
with open(r'/Users/dl/Documents/GitHub/Turbopump/TCA/TCA_params.yaml') as file: ##CHANGED FOR DANIEL
		tca_params = yaml.safe_load(file)

In [129]:
# == Conversion between units ==
psi_into_pa = 6894.76 # Convert psi -> Pascal
meters_into_inches = 39.37 # Convert meters -> inches
degrees_into_rad = np.pi/180 # Convert degrees -> radians
lbm_into_kg = 0.453592 # Convert lbm -> kg
lbf_into_newton = 4.44822 # Convert lbf -> Newton
g0 = 9.81 # m/s^2

In [130]:
# == Desing Parameters (Change as needed) == 
mdot = tca_params['tca_throat_choked_flow']*lbm_into_kg #Total propellant mass flow rate [kg/s]
OF_Ratio = tca_params['oxidizer_fuel_ratio'] #Mixture ratio O/F 
rho_rp1 = 810 #RP1 Density [kg/m^3] at injector conditions
rho_lox = 1141 #LOX Density [kg/m^3] at injector conditions


Pc = tca_params['tca_chamber_pressure']*psi_into_pa #Chamber stagnation pressure [Pa]
Pin = tca_params['incoming_pressure']*psi_into_pa #injector inlet pressure [Pa]

delta_P_ox = 0.40*Pc #Pressure drop across injector [Pa]
delta_P_rp1 = 0.40*Pc #Pressure drop across injector [Pa]

Cd_ox = tca_params['discharge_coefficient_lox'] #Discharge coefficient
Cd_rp1 = tca_params['discharge_coefficient_rp1'] #Discharge coefficient

Length_chamber = tca_params['tca_chamber_length']/meters_into_inches #Combustion chamber length [m]
CombDiam = tca_params['tca_chamber_diameter']/meters_into_inches #chamber inner diameter [m]
throat_diameter = tca_params['tca_throat_diameter']/meters_into_inches #throat diameter [m]

num_holes_rp1_inj = tca_params['injector_number_of_holes_rp1'] #number of RP1 holes in injector faceplate
num_holes_lox_inj = tca_params['injector_number_of_holes_lox'] #number of LOX holes in injector faceplate

# == Injector Geometry Parameters ==
wall_thickness_rp1 = tca_params['injector_plate_thickness_rp1'] #[m]
wall_thickness_ox = tca_params['injector_plate_thickness_ox'] #[m]

impinge_fraction = tca_params['impinge_fraction'] #streams will impinge at 2.2% of chamber length

#MANIFOLD DESIGN
# Allow separate manifold heights for LOX and fuel. Fall back to the
h_manifold_ox = tca_params['h_manifold_ox']

dpFracMani = 0.09 #dp manifold ~9% of injector dp

#Friction factors and local loss coefficients
fOx = 0.01 #darcy friction factor LOX
fRP1 = 0.01 #darcy friction factor RP1
KOx = 1 #lumped local loss K LOX
KRP1 = 1 #lumped local loss K RP1
NinletsOx = 1 #number of LOX manifold inlets
NinletsRP1 = 1 #number of RP1 manifold inlets

In [131]:
mdot_kero = mdot/(1+OF_Ratio) #Fuel mass flow (RP-1)
mdot_lox = mdot*OF_Ratio/(1+OF_Ratio) #Oxidizer mass flow (LOX)

This function distribute the total propellant mass flow among a specific set of injector holes
    mdot_propellant --- the total mass flow rate of that propelant [kg/s]
    number_holes --- number of holes of this specific type 
    total_number_of_holes --- total number of holes that share the propellant flow

In [132]:
def mdot_prop(mdot_propellant, number_holes, total_number_of_holes):
    mdot_propel = mdot_propellant * number_holes/(total_number_of_holes)
    mdot_per_hole = mdot_propel/number_holes
    return mdot_propel, mdot_per_hole

mdot_rp1_inj, mdot_rp1_inj_per_hole = mdot_prop(mdot_kero, num_holes_rp1_inj, num_holes_rp1_inj)
mdot_lox_inj, mdot_lox_inj_per_hole = mdot_prop(mdot_lox, num_holes_lox_inj, num_holes_lox_inj) 

print(f"RP1 mass flow: {mdot_rp1_inj:.3f} kg/s")
print(f"LOX mass flow: {mdot_lox_inj:.3f} kg/s")

RP1 mass flow: 2.909 kg/s
LOX mass flow: 6.110 kg/s


This function computes the required injector orifice diameter for a given mass flow 
    mdot_per_prop --- mass flow rate through this specific parth [kg/s]
    rho --- propellant density at injector conditions
    delta_pressure --- pressure drop across the orifice [psi]
    number_holes --- number of identical holes sharing this mass flow

In [133]:
def diameter(mdot_per_prop, rho, delta_pressure, number_holes,Cd): 
    #following the orifice flow model, and solving for the Area
    Total_area = mdot_per_prop/(Cd*np.sqrt(2*rho*delta_pressure))
    Area_per_hole = Total_area/number_holes
    d_hole = 2*np.sqrt(Area_per_hole/np.pi) 
    return d_hole

diameter_inj_rp1 = diameter(mdot_rp1_inj, rho_rp1, delta_P_rp1, num_holes_rp1_inj,Cd_rp1)
diameter_inj_lox = diameter(mdot_lox_inj, rho_lox, delta_P_ox, num_holes_lox_inj,Cd_ox)

print(f"RP1 injector hole diameter: {diameter_inj_rp1*10**3:.3f} mm")
print(f"LOX injector hole diameter: {diameter_inj_lox*10**3:.3f} mm")


RP1 injector hole diameter: 1.166 mm
LOX injector hole diameter: 1.552 mm


This function computes the ideal jet exit velocity through an injector orifice using Bernoulli's incompressible flow relation with a pressure drop
    delta_pressure_injector --- pressure drop across the injector 
    rho --- propellant density at injector conditions [kg/m^3]

In [134]:
def velocity(delta_pressure_injector, rho):
    velocity = np.sqrt((2*delta_pressure_injector)/rho)
    return velocity

v_lox = velocity(delta_P_ox, rho_lox)
v_rp1 = velocity(delta_P_rp1, rho_rp1)

print(f"LOX velocity: {v_lox:.2f} m/s")
print(f"RP1 velocity: {v_rp1:.2f} m/s")

LOX velocity: 49.16 m/s
RP1 velocity: 58.35 m/s


This function determines the oxidizer injection angle (theta_ox) for a doublet impinging injector such that the post-impingement spray sheet forms a desired angle beta with respect to the chamber axis, for a given fuel injection angle theta_fuel. 

In [135]:
def solve_thetas(mdot_fuel, mdot_ox, v_fuel, v_ox, theta_fuel, beta_target_deg, 
                 theta_ox_min_deg=5.0, theta_ox_max_deg=80.0, dtheta_deg=0.05):
    theta_f_rad = theta_fuel*degrees_into_rad
    beta_target = beta_target_deg*degrees_into_rad
    p_f = mdot_fuel * v_fuel
    p_ox = mdot_ox * v_ox
    best_err = 1e12
    best_theta_ox_deg = None
    best_beta_deg = None 
    theta_ox_deg = theta_ox_min_deg
    while theta_ox_deg <= theta_ox_max_deg:
        theta_ox_rad = theta_ox_deg*degrees_into_rad
        px = p_ox*np.sin(theta_ox_rad) - p_f*np.sin(theta_f_rad)
        py = p_ox*np.cos(theta_ox_rad) + p_f*np.cos(theta_f_rad)
        R = px/py
        beta_res = np.arctan(R)
        err = abs(beta_res - beta_target)
        if err < best_err:
            best_err = err
            best_theta_ox_deg = theta_ox_deg
            best_beta_deg = beta_res/degrees_into_rad
        theta_ox_deg += dtheta_deg
    return best_beta_deg, best_theta_ox_deg

These functions compute the geometric relationships of a doublet impinging injection. Including Manifold Design

In [136]:
def compute_spacing_doublet(Lc, impingement_fraction, theta_fuel_deg, theta_ox_deg):
    #compute axial impingement location
    z_imp = Lc *impingement_fraction
    #converting jet angles into radians
    theta_f = theta_fuel_deg*degrees_into_rad
    theta_ox = theta_ox_deg*degrees_into_rad
    #Lateral displacement of each jet at the impingement plane:
    # x = z * tan(theta)
    d_imp_f = z_imp * (np.tan(theta_f))
    d_imp_ox = z_imp * (np.tan(theta_ox))
    #required injector-to-injector spacing for the jets to meet at z_imp
    d_fo = z_imp * (np.tan(theta_f) + np.tan(theta_ox))
    return z_imp, d_imp_f, d_imp_ox, d_fo

In [137]:
def dist(thickness, theta):
    theta_rad = theta*degrees_into_rad
    dist = thickness*(np.tan(theta_rad))
    return dist

In [138]:
def design_manifold(d_orifice, n_orifices_total, h, rho, m_dot_total, n_inlets=1):
    # compute orifices and flow handled by a single inlet/branch
    n_orifices = max(1, int(n_orifices_total / max(1, n_inlets)))
    m_dot = m_dot_total / max(1, n_inlets)

    # Total exit area for the orifices served by this inlet:
    A_exit = n_orifices * (np.pi*(d_orifice**2)/4.0)
    # required manifold cross-sectional area (4:1 rule -> manifold area = 4 * exit area)
    req_manifold_area = 4.0 * A_exit
    # geometry with rounded corners (choose corner radius r = h/4)
    r = h/4.0
    corner_area_loss = (r**2)*(4 - np.pi)
    w = (req_manifold_area + corner_area_loss)/h
    # manifold internal velocity (for this inlet/branch)
    v = m_dot/(rho*req_manifold_area) if req_manifold_area>0 else np.inf
    return A_exit, req_manifold_area, h, w, r, v

In [144]:
def design_manifold_circular(d_orifice, n_orifices_total, rho, m_dot_total, n_inlets=1):
    # orifices and flow handled by a single inlet/branch
    n_inlets = max(1, int(n_inlets))
    n_orifices = max(1, int(n_orifices_total / n_inlets))
    m_dot = m_dot_total / n_inlets

    # total exit area for the orifices served by this inlet
    A_exit = n_orifices * (np.pi * (d_orifice**2) / 4.0)

    # 4:1 rule: manifold area = 4 * total orifice exit area
    A_manifold = 4.0 * A_exit

    # circular cross-section: A = pi D^2 / 4  -> D = sqrt(4A/pi)
    D = np.sqrt(4.0 * A_manifold / np.pi) if A_manifold > 0 else np.inf
    R = 0.5 * D if np.isfinite(D) else np.inf

    # internal velocity in this branch
    v = m_dot / (rho * A_manifold) if A_manifold > 0 else np.inf

    return A_exit, A_manifold, D, R, v

A_exit, A_manifold, Circular_mani_diameter, R, v = design_manifold_circular(
    d_orifice=tca_params['injector_hole_diameter_rp1']/1000,
    n_orifices_total=8,
    rho=rho_rp1,
    m_dot_total=0
)

print(f"RP1 manifold diameter: {Circular_mani_diameter*1000:.3f} mm")

RP1 manifold diameter: 6.998 mm


Computes the LOX injection angle rquired so that the resultant post-impingement sheet angle matches the desired beta.

In [140]:
theta_rp1_deg = float(input("Fuel angle (deg): "))
beta_des = float(input("Enter an angle for the impingement (relative to vertical): "))
beta_res_deg, theta_lox_deg = solve_thetas(mdot_rp1_inj_per_hole, mdot_lox_inj_per_hole, v_rp1, v_lox, theta_rp1_deg, beta_des)

print(f"Oxidizer angle: {theta_lox_deg:.3f} deg")
print(f"Fuel angle: {theta_rp1_deg:.3f} deg")  
print(f"Resulting impingement angle: {beta_res_deg:.3f} deg")


Oxidizer angle: 30.000 deg
Fuel angle: 40.000 deg
Resulting impingement angle: 6.008 deg


Impinging geometry check. Calculate impingement geometry using the final RP-1 and LOX angles:

In [141]:
z_imp, d_rp1, d_lox, d_fo_req = compute_spacing_doublet(Length_chamber, impinge_fraction, theta_rp1_deg, theta_lox_deg)

print(f"zimp: {z_imp*10**3:.3f} mm")
print(f"Required injector-to-injector spacing: {d_fo_req*1000:.3f} mm")


zimp: 5.548 mm
Required injector-to-injector spacing: 7.858 mm


Distance between inner and outer orifices

In [142]:
dist_fuel = dist(wall_thickness_rp1, theta_rp1_deg)
dist_ox = dist(wall_thickness_ox, theta_lox_deg)

print(f"Fuel lateral displacement through plate thickness: {dist_fuel*1000:.3f} mm")
print(f"Oxidizer lateral displacement through plate thickness: {dist_ox*1000:.3f} mm")


Fuel lateral displacement through plate thickness: 5.035 mm
Oxidizer lateral displacement through plate thickness: 5.196 mm
